Funzioni che sono necessarie per la corretta estrazione e organizzazione delle risposte del modello

In [ ]:
import re


# Questa funzione si occupa di ottenere le entità estratte dal modello e di organizzarle in un dizionario

def extract_entities_with_categories_from_prediction(sentence):
  pattern = re.compile(r'(\w+):\s*([^;]+)')
  matches = pattern.findall(sentence)
  result_dict = {}
  for match in matches:
      key = match[0].strip()
      values = match[1].strip()
      if(key in result_dict.keys()):
        result_dict[key].append(values)
      else:
        result_dict[key] = []
        result_dict[key].append(values)
  return result_dict

# Questa funzione si occupa di organizzare le entità che sono presenti nel dataset in un dizionario
def extract_entities_with_categories_from_data(data):
  keys = ["Metric", "Method", "Material", "Task", "OtherScientificTerm", "Generic"]
  result_dict = {}
  for key in keys:
    sentences = data[key]
    if(len(sentences) > 0):
      result_dict[key] = sentences
  return result_dict


Funzione che restituisce la valutazione fatta per span comprensiva di tipo 

In [ ]:
import json

# Questa funzione si occupa di calcolare le metriche di precision, recall e f1 score per le entità estratte dal modello

def scores_span(path_result,path_dataset):

  def evaluate_ner(ground_truth, predicted):
      # Get unique entity types
      entity_types = set(ground_truth.keys()).union(predicted.keys())

      # Inizializzo le variabili per i true positives, false positives e false negatives
      true_positives = 0
      false_positives = 0
      false_negatives = 0
       
      # Per ogni tipo di entità calcolo i true positives, false positives e false negatives

      for entity_type in entity_types:
          true_positives += sum(1 for entity in ground_truth.get(entity_type, [])
                                if entity in predicted.get(entity_type, []))
        
          false_positives += sum(1 for entity in predicted.get(entity_type, [])
                                if entity not in ground_truth.get(entity_type, []))

          false_negatives += sum(1 for entity in ground_truth.get(entity_type, [])
                                if entity not in predicted.get(entity_type, []))

      return true_positives, false_positives, false_negatives

  true_positives = 0
  false_positives = 0
  false_negatives = 0
  i = 0
  data = []
  
  # Salvo le entità estratte dal modello che erano presenti nel file delle predizioni
  with open(path_result) as f:
      for line in f:
          data.append(json.loads(line)["prediction"])
  # Per ogni riga del dataset calcolo le metriche
  with open(path_dataset) as f:
      for line in f:
          data2 = json.loads(line)
          result = data[i]
          predicted_output = extract_entities_with_categories_from_prediction(result)
          result_dict = extract_entities_with_categories_from_data(data2)
          aux1, aux2, aux3 = evaluate_ner(result_dict,predicted_output)
          true_positives += aux1
          false_positives += aux2
          false_negatives += aux3
          i += 1

  precision = true_positives / (true_positives + false_positives)
  recall = true_positives / (true_positives + false_negatives)
  f1_score = 2 * (precision * recall) / (precision + recall)
  print("Precision: " + "{:.2f}".format(precision * 100) + "%")
  print("Recall: " + "{:.2f}".format(recall * 100) + "%")
  print("F1 Score: " + "{:.2f}".format(f1_score * 100) + "%")

# Esempio di utilizzo, in questo caso vengono calcolate le metriche per tutti i tipi di dataset
#print("\nNo incapsulamento")
#scores_span("/content/drive/MyDrive/Tesi/result/result_flan_no_inc_15.json","/content/drive/MyDrive/Tesi/dataset/scierc_test_no_inc.json")
#print("\nCon incapsulamento")
#scores_span("/content/drive/MyDrive/Tesi/result/result_flan_inc_15.json","/content/drive/MyDrive/Tesi/dataset/scierc_test_inc.json")
#print("\nNo differenze")
#scores_span("/content/drive/MyDrive/Tesi/result/result_flan_no_diff_15.json","/content/drive/MyDrive/Tesi/dataset/scierc_test_no_diff.json")
